In [ ]:
from recognize_image.treasureAreaDetect import TreasureAreaDetect
from recognize_image.minifig import Minifig

In [ ]:
class Main:
    def __init__(self):
        pass
    def execute(self):
        pass

In [ ]:
if __name__ == "__main__":
    main = Main()
    main.execute()